In [4]:
import numpy as np
import pandas as pd 
import pickle
import csv
from PIL import Image
import os
import random
import shutil

from matplotlib.pyplot import imread
from PIL import Image
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator

In [2]:
csv_path = "/datasets/RTSD/full-gt.csv"
photo_path = "/datasets/RTSD/rtsd-frames/"

target_path = "/home/linnik/Datasets/Signs/RTSD_crop_all/"

In [3]:
csv = pd.read_csv(csv_path)
csv.head()

,filename,x_from,y_from,width,height,sign_class,sign_id
0,autosave01_02_2012_09_13_33.jpg,649,376,18,18,2_1,0
1,autosave01_02_2012_09_13_34.jpg,671,356,20,21,2_1,0
2,autosave01_02_2012_09_13_35.jpg,711,332,27,26,2_1,0
3,autosave01_02_2012_09_13_36.jpg,764,290,37,36,2_1,0
4,autosave01_02_2012_09_13_36.jpg,684,384,17,17,1_23,1


In [12]:
%%time

for i in range(len(csv)):
        image_name = csv.loc[i]["filename"]
        class_id = csv.loc[i]["sign_class"]

        img = Image.open(photo_path + image_name)
        area = (int(csv.loc[i]["x_from"]), int(csv.loc[i]["y_from"]), int(csv.loc[i]["x_from"]) + int(csv.loc[i]["width"]), int(csv.loc[i]["y_from"]) + int(csv.loc[i]["height"]))
        cropped_img = img.crop(area)

        if os.path.isdir(target_path + class_id):
            cropped_img.save(target_path + class_id + "/" + image_name[:-4] + str(i) + ".jpg")
        else:
            os.mkdir(target_path + class_id)
            cropped_img.save(target_path + class_id + "/" + image_name[:-4] + str(i) + ".jpg")

CPU times: user 23min 55s, sys: 28.4 s, total: 24min 24s
Wall time: 45min 47s


In [5]:
n_classes = len(os.listdir(target_path))
print(n_classes)

198


### 10% to test

In [38]:
data_path = "/home/linnik/Datasets/Signs/temp/"
target_path = "/home/linnik/Datasets/Signs/12_Classes_NAMI+RTSD/test/"

In [39]:
for folder in os.listdir(data_path):
    num_photos = len(os.listdir(data_path + folder))
    while len(os.listdir(data_path + folder)) > 0.9 * num_photos:
        photo = random.choice(os.listdir(data_path + folder))
        shutil.move(data_path + folder + "/" + photo, target_path + folder + "/" + photo)

### 2500 to test other (folder "10")

In [44]:
black_list = ['1_22', '1_23', '2_4', '2_5', '3_1', '3_2', '3_4_1', '3_4_n2', '3_4_n5', '3_4_n8',
              '3_24_n5', '3_24_n10', '3_24_n20', '3_24_n30', '3_24_n40', '3_24_n50', '3_24_n60', '3_24_n70',
              '3_24_n80', '3_25_n20', '3_25_n40', '3_25_n50', '3_25_n70', '3_25_n80', '3_31', '5_19_1']

In [60]:
data_path = "/home/linnik/Datasets/Signs/RTSD_crop_all/"
target_path = "/home/linnik/Datasets/Signs/12_Classes_NAMI+RTSD/test/10/"

while len(os.listdir(target_path)) < 2500:
    for folder in os.listdir(data_path): 
        if folder not in black_list:
            photo = random.choice(os.listdir(data_path + folder))
            shutil.copy(data_path + folder + "/" + photo, target_path + photo)

### 5000 to train other (folder "10")

In [61]:
black_list = ['1_22', '1_23', '2_4', '2_5', '3_1', '3_2', '3_4_1', '3_4_n2', '3_4_n5', '3_4_n8',
              '3_24_n5', '3_24_n10', '3_24_n20', '3_24_n30', '3_24_n40', '3_24_n50', '3_24_n60', '3_24_n70',
              '3_24_n80', '3_25_n20', '3_25_n40', '3_25_n50', '3_25_n70', '3_25_n80', '3_31', '5_19_1']

In [62]:
data_path = "/home/linnik/Datasets/Signs/RTSD_crop_all/"
target_path = "/home/linnik/Datasets/Signs/12_Classes_NAMI+RTSD/train/10/"

while len(os.listdir(target_path)) < 6000:
    for folder in os.listdir(data_path): 
        if folder not in black_list:
            photo = random.choice(os.listdir(data_path + folder))
            shutil.copy(data_path + folder + "/" + photo, target_path + photo)

### 6000 to train (folder "09")

In [65]:
data_path = "/home/linnik/Datasets/Signs/temp/09/"
target_path = "/home/linnik/Datasets/Signs/12_Classes_NAMI+RTSD/train/09/"

while len(os.listdir(target_path)) < 6000:
    photo = random.choice(os.listdir(data_path))
    shutil.move(data_path + photo, target_path + photo)

### Sings 4.6 and 4.7 augmentation

In [182]:
path = "/home/linnik/Datasets/Signs/temp/"
aug_path = "/home/linnik/Datasets/Signs/4.7_aug/"

img_height, img_width = 200, 200

In [183]:
datagen = ImageDataGenerator(
    rescale = 1.0 / 255,
    zoom_range = 0.5,
    rotation_range = 15,
    width_shift_range = img_width // 5,
    height_shift_range = img_height // 5,
    horizontal_flip = False,
    vertical_flip = False)

#datagen = ImageDataGenerator(rescale = 1.0 / 255)

In [184]:
generator = datagen.flow_from_directory(
    path,
    shuffle = False, 
    target_size = (img_height, img_width),
    batch_size = 1,
    class_mode = 'categorical')

Found 26 images belonging to 1 classes.


In [185]:
i = 68

for x, y in generator:
    plt.imsave(aug_path + str(i) + ".jpg", x[0])
    i += 1
    if i > 400:
        break